# Text Classification

## 0. Prerequisites

This notebook assumes that you have a basic command of the Python programming language. If you have no experience with Python, there are innumerable great tutorials and introductions out there. You can find a good overview of some of them [here](https://wiki.python.org/moin/BeginnersGuide/Programmers).

It also assumes that you have gone through following sessions in the [Text Analysis with Python series](https://git.dartmouth.edu/lib-digital-strategies/RDS/workshops/text-analysis/text-analysis-with-python):
- Strings and Files
- Word counts
- TF-IDF

Finally, we will use the machine learning toolkit `scikit-learn` and assume that you have gone through the [Intro to Machine Learning with scikit-learn](https://git.dartmouth.edu/lib-digital-strategies/RDS/workshops/machine-learning/intro-to-machine-learning-with-scikit-learn).

## 1. Introduction

Let's say you are dealing with a number of different pieces of writing and want to categorize them into a set of predefined groups. For example, you might want to find out the language a particular text is written in. Or you are dealing with an email at work and are trying to figure out which department to forward it to. In text classification, you assign a *class* to each piece of text. So, for example, you assign the class `Czech`, `Dutch`, `English`, or `Polish` to a document, or the class `accounting` or `customer service` to the email.

<div class="alert alert-block alert-info"> 

How would you, as a human, solve these examples? Think about it step-by-step: How would you mentally process the text, what would inform your decision?
</div>

We can do this manually for smaller amounts of text, but we of course quickly run into problems at a larger scale: You would not want to classify every single incoming email at Dartmouth as `spam` or `no spam`, of course. This is where algorithmic text classification using machine learning can help!

Text Classification is a three-step process:
1. Extract descriptive features from a sufficiently large number of texts belonging to known categories/classes
2. Train a classifier using these features to discriminate between these classes
3. Use the trained classifier to classify new text pieces

In this session, we will walk through this process by building up a classifier that can tell us if any given State of the Union address was delivered by a Republican or a Democratic president. If you are running this notebook on Dartmouth's JupyterHub, the dataset is already available to you under `~/shared/RR-workshop-data/state-of-the-union-dataset/txt`. Otherwise you can download the dataset [here](https://git.dartmouth.edu/lib-digital-strategies/RDS/datasets/state-of-the-union-dataset/-/archive/main/state-of-the-union-dataset-main.zip) and put the in a folder of your choosing.

<div class="alert alert-block alert-info"> 

**Caveat emptor:** This particular system we are building here is most likely not the optimal system for this task. There are literally thousands of models and algorithms we could choose from and even more feature sets we could consider. The main purpose of this notebook, however, is to give you a relatively simple example that will hopefully give you a good idea of how text classification works *in principle*. Maybe you even feel inspired to engineer your own features or try out different classifiers?

If you do, please [let us know how it went](mailto:simon.stone@dartmouth.edu?subject=Text%20classification%20workshop)!
</div>

As mentioned above, the examples in our training set need to be already labeled (i.e., marked as *Republican* or *Democrat*). Fortunately, the State of the Union dataset includes some meta information we can use to do that. Since this kind of processing is outside the scope of this notebook, we moved this task to a separate notebook `add-meta.ipynb`. If you are interested, you can open that notebook and see how it works excactly, but for now we will simply run that other notebook using a [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [ ]:
%run add-meta.ipynb

<div class="alert alert-block alert-info"> 

**Note:** If your State of the Union dataset is not at the default location (e.g., you are not working on Dartmouth's JupyterHub), you may have to open `add-meta.ipynb` and change the variable `dataset_folder`  accordingly!
</div>

That notebook produced a CSV file we can now read using pandas:

In [ ]:
import pandas as pd

sotu = pd.read_csv('data/sotu-extended.csv')
# Look at five random samples from the dataframe
sotu.sample(5)

## 2. Feature Extraction

### 2.1 Numeric features and encoding

The goal of feature extraction is to *define* and *extract* features that hopefully best help to distinguish the classes of interest. Basically anything that describes the content of the analyzed texts can be a feature, *as long as you can express it as a number*. 

This limitation is imposed by the fact that most machine learning algorithms work with numeric computations and have no way of dealing with more abstract concepts like "meaning" or "context".

We already talked about some numeric features in previous sessions: 
- word count
- word frequencies
- TF/IDF

But even features that are not immediately numeric can be expressed in numbers (i.e., *encoded*). Let's take for example the feature *emotion*. Intuitively, this feature would have some descriptive levels like *sad*, *happy*, or *angry*. But we could also express these as numbers, if we mark the presence of one of these emotions in a text using `1` (for present) and `0` (for absent):


<style type="text/css" >
table {
    border-collapse: collapse;
    text-align: center;
    border-top: 3px solid;
    border-bottom: 3px solid;
}

tr, td, th {
    border-bottom: none !important;
    border-left: none !important;
    border-right: none !important;
}

</style>

<table>
  <tr>
    <th>Text</th>
    <th>Sad</th>
    <th>Happy</th>
    <th>Angry</th>
  </tr>
  <tr>
    <td>"My favorite show has been cancelled."</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
  </tr>  
  <tr>
    <td>"I got a promotion at work."</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
  <td>"My phone battery died just when I needed it the most."</td>
  <td>0</td>
  <td>0</td>
  <td>1</td>
  </tr>
  <tr>
    <td>"I was happy to see my childhood home one last time before it was sold.</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
  </tr>
</table>
  


<div class="alert alert-block alert-info"> 

**Note:** We can even express mixed emotions this way! Just take a look at the last example sentence.
</div>

### 2.2 Document- versus sentence-level features

The next important thing to consider when you want to extract features is the *level* at which they are extracted.

So far, we have only extracted features at a *document level*: Counting the words in the entire text, determmining the word frequencies in the entire text, and so on.

But a *document* consists of *paragraphs*, a *paragraph* consists of *sentences*, and a *sentence* consists of *words*. You could even find more units, like clauses or even letters.

So almost every unit of text is actually a sequence of smaller units. You could consider extracting features at any of these levels. For example, you could extract the word count in every sentence instead of for an entire document.

The *representation* of each document in that case also becomes a sequence. For example:

In [ ]:
from collections import Counter

a_text = 'This is a text about cats and dogs . But not about bunnies . I cannot stress enough , how little this text is about bunnies . I like bunnies , but this is not about them.'

# Document-level feature extraction
word_frequencies = Counter(a_text.lower().split()).most_common()


print("At a document-level, we observe the following features:")
print(f"{word_frequencies = }")

<div class="alert alert-block alert-info"> 

As you can see, document-level features lose all the structure and context inherent in a document. The basic assumption here is that the relative order of the words does not matter for the intended purposes. This perspective on a text is therefore often called the *bag-of-words* model.
</div>

So if we look at this text at a document level, we might conclude that it is about bunnies! The word bunnies appears 3 times, after all. But if we look at it at a sentence level:

In [ ]:
print("At a sentence level, we observe the following sequence of features:")
for idx, sentence in enumerate(a_text.split('.')):
    print(f'Sentence {idx}:')
    print(Counter(sentence.lower().split()).most_common())


Now we see that the word `bunnies` always occurs together with some form of `not` in the same sentence. So maybe this text is not about bunnies, after all?

<div class="alert alert-block alert-info"> 

This (somewhat crude) example demonstrates that sentence-level features are much better at capturing *local context*. However, since sentence-level features are more complex to process (see below), there are other techniques like N-grams and collocations that try to achieve the same thing while still remaining at the document level.
</div>

Knowing this, you might be tempted to always go with sentence-level features. However, where a document-level approach converts a document into one set of features (a *feature vector*), a sentence-level approach converts the document into a *sequence* of feature vectors. The problem here is that conventional machine learning models can only process fixed-size feature vectors, not sequences of them. Neural networks, on the other hand, can be very good at handling sequences, which is why Large Language Models (like ChatGPT) are so good at what they do. 

Using neural networks, though, comes with many challenges regarding the complexity of the models involved and, above all, the amount of data needed to train them. For many tasks, it is therefore advisable to follow [the principle of parsimony](https://en.wikipedia.org/wiki/Occam's_razor) and use document-level features. They can still get the job done!

### 2.3 Extracting features from the State of the Union addresses



Feature extraction can be a very explorative process full of trial and error. To keep things simple, we will use the TF-IDF of all nouns in the State of the Union addresses.

We will first clean up the raw speeches (as we have learned in previous sessions of this series) before discarding everything but the nouns and calculating the TF-IDF for them.

Let's get rid of all punctuation:

In [ ]:
import string

print('Replace all occurrences of these punctuation marks: ', string.punctuation)
for symbol in string.punctuation:
    sotu['Text'] = sotu['Text'].str.replace(symbol, ' ', regex=False)
   
sotu

Next, we tokenize each speech. We can use a simple [WhitespaceTokenizer](https://www.nltk.org/api/nltk.tokenize.regexp.html#nltk.tokenize.regexp.WhitespaceTokenizer) here:

In [ ]:
from nltk.tokenize import WhitespaceTokenizer

sotu['Tokens'] = sotu['Text'].str.lower().apply(WhitespaceTokenizer().tokenize)
sotu

Now we will remove the stopwords:

In [ ]:
from nltk import corpus

stopwords = corpus.stopwords.words('english')

def remove_stopwords(text):
    return [word for word in text if word not in stopwords]

sotu['Tokens w/o stopwords'] = sotu['Tokens'].apply(remove_stopwords)
sotu


Finally, we can extract the nouns. We will use the [pos_tag](https://www.nltk.org/api/nltk.tag.pos_tag.html#nltk-tag-pos-tag) function, which by default uses the [Penn Treebank tag set](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). Here is what that function's output looks like for a couple of tokens:

In [ ]:
from nltk import pos_tag

for tokens in sotu['Tokens w/o stopwords'].sample(1):
    print(pos_tag(tokens[:10]))

<div class="alert alert-block alert-info"> 
The POS tagger makes quite a few mistakes which is mostly due to the fact that we are trying to tag each individual word. It usually works better if you pass an entire sentence so it can better exploit the context. For simplicity, though, we will simply accept the errors here.
</div>

Notice that the noun tags all include the letters `'NN'`. We can therefore extract all the nouns by checking for that specific letter combination in the tag:

In [ ]:
def extract_nouns(x):
    tags = pos_tag(x)
    words = list()
    for word, pos in tags:
        if 'NN' in pos:
            words.append(word)
    return words


sotu['Nouns'] = sotu['Tokens w/o stopwords'].apply(extract_nouns)
sotu

We have converted the full text of each State of the Union address into a collection of nouns used in that speech. Great! Now we can use `scikit-learn`'s [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) to calculate our features. Before we do that, however, we need to split our dataset into a training and test set.

<div class="alert alert-block alert-info"> 

In any machine learning task, you want to set some data aside to check if the model can generalize what it has learned on the training set to new, *unseen* data. If we did not do that in this example, we would to wait until the next State of the Union address to find out how well our model really works!
</div>


For many datasets, you can simply randomly divide all your data and set a test set aside (usually 10 % of the entire dataset). In this particular case, however, we have one complication: Each president gave several speeches! If we would put speeches from the same president into the training *and* the test set, the model will probably pick up on the similarities between those speeches because they are by the same *speaker* (and/or speech writer) rather than because they are by the same *party*.

To avoid this, we group the speeches by president and then split along these groups using `scikit-learn`'s [`GroupShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html#sklearn.model_selection.GroupShuffleSplit):

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

# We will only use two parties to keep things simple
subset = sotu.query("Party == 'Democratic' | Party == 'Republican'")

# Split the speeches into subsets for training (90 %) and testing (10 %)
splits = GroupShuffleSplit(n_splits=1,                      # Make one split (i.e., create two subsets)
                           test_size=0.1,                   # Make the testing subset 0.1 times the size of the training subset
                           random_state=23                  # This is here so our split is always the same. Usually you would not set this to get a truly random split.
                           ).split(subset,                  # The dataset to be split
                                   groups=subset['Name'])   # The grouping variable to use. This makes sure no president shows up in both training and testing set!

# The output above just gives us the indices of the training and testing samples:
for train_idx, test_idx in splits:
    sotu_train = subset.iloc[train_idx]
    sotu_test = subset.iloc[test_idx]

In [ ]:
sotu_train['Name'].unique()

In [ ]:
sotu_test['Name'].unique()

From now on, our feature extractor and classifier are only allowed to use `sotu_train`! Otherwise it would be like they get to cheat by looking at the solution.

We can now create our feature extractor and calculate the TF-IDF for all nouns in the training set:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


feature_extractor = TfidfVectorizer(input='content',            # This lets the TfidfVectorizer know that we will pass it a sequence of items to vectorize
                                    preprocessor=lambda x: x,   # We did the preprocessing ourselves, so just pass everything through
                                    tokenizer=lambda x: x,      # We also did the tokenization ourselves
                                    token_pattern=None,         # Since we do not tokenize, we can avoid a warning by setting this to None
                                    max_features=500,           # We will only use the 500 most salient words (highest term frequency) from the entire vocabulary
                                    ngram_range=(1, 2),         # We will use unigrams and bigrams
                                    )

# Extract the features!
features = feature_extractor.fit_transform(sotu_train['Nouns'])

The calculate features by themselves are difficult to interpret. But it may be interesting to look at the words that were kept (i.e., the Top 500 words by term-frequency):

In [ ]:
feature_extractor.get_feature_names_out()

Instead of trying to interpret these ourselves to distinguish between Democratic and Republican speeches, we will use a classifier to do the work for us!

## 3. Classification

As in any machine learning model, there is a vast number of classifiers to choose from. There are even flowcharts like [this one](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) that are supposed to help you make the right choice.

The bitter truth is, however, that only trial and error can truly tell you what the optimal classifier for your particular problem is. It is therefore often advisable to set some expectations, like setting a target performance level that is considered "good enough" instead of chasing perfection. Or you might want to emphasize the interpretability of the results and the transparency of the decision making process over the raw accuracy of the predictions.

For this example, we want to do the latter and therefore choose a model that offers excellent insights into the rules it constructs from the data to come to a conclusion: The decision tree.

### 3.1 Decision Tree classifiers

<div class="alert alert-block alert-info"> 

Think of the game "20 questions", where another player thinks of something (e.g., a particular animal) and you need to guess it by asking a series of Yes or No questions. How do you choose the first question you ask? How do you choose the next questions? When do you make you first guess?
</div>

If you track the flow of a couple of rounds of 20 questions, it would probably look something like this chart: 

<img src='img/20-questions-tree.svg' style='width:40%'/>

An optimal strategy would be to pick the question that gives you *the most information* at every stage. In this context, *the most information* would mean the greatest reduction of options. So you would probably ask questions that exclude as many animals as possible as you zero in on the correct answer!

Decision trees work in a very similar way. They classify data by "asking" a series of Yes or No "questions". These questions are comparisons of the value of a specific feature with some kind of threshold value. Depending on the result of this comparison, some other feature is compared to some other value. A trained decision tree therefore looks a lot like the flow chart above! 

Let's train a decision tree on our features here and see what sort of "questions" it comes up with!

First, we need to create a fresh, untrained decision tree:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(random_state=23)  # Training the tree involves some randomness. By setting the initial random state, here, we can make sure that we get the same results every time we run this notebook.

Now we can train the decision tree to discover the "questions" that best help us classify the speeches' party affiliation:

In [ ]:
classifier.fit(features, sotu_train['Party'])

We look at this tree using `scikit-learn`'s [`plot_tree()`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) function. To make things even more readable, we pass the names of the features and the classes, as well as set the figure and font size to suitable values.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

feature_names = feature_extractor.get_feature_names_out()
class_names = classifier.classes_

plt.figure(figsize=(22,15))
plot_tree(classifier, feature_names=feature_names, class_names=class_names, fontsize=10);

Each box has the following structure:

- The first line is the "question" that is being asked (e.g. "Does the term `commission` have a term-frequency of less than 0.035?").
- `gini` denotes the [Gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) at this stage, which indicates how sure you can be to make decision at this stage. A value of 0.5 indicates a 50 % chance of assigning the wrong label if you were to base your decision just on the questions asked so far and lower values are better (ideally 0).
- `samples` tells you how many observations from the training set (so in our example how many SOTU speeches) are still undecided at this stage.
- `value` lists the number of observations in each class. In our example, the first entry is the number `Democratic` speecges, the second entry is the number of `Republican` speeches.
- `class` denotes the most frequent class observed at this stage in the training set and thus the classification result *if you were to ask no further questions*, i.e. would stop traversing the tree at this stage.

Just like in the optimal "20 questions" strategy, the devision tree starts with the "questions" (called *splits*) that add the most amount of information. So by looking at the boxes at the top of the tree, you can get a sense of which terms are the most important to tell `Republican` from `Democratic` speeches.


<div class="alert alert-block alert-info"> 

Since the training involves a certain amount of randomness, the splits that are made by the Decision Tree can be different every time you train it. You should therefore always repeat the training with several different initial random states. Another way to check the validity of the rules that were found is to apply them to the test set and see how well they hold up on unseen data!
</div>

To test the decision tree, we first extract features from the test set using the same feature extractor as before:

In [ ]:
# Only transform the test data, do not re-fit the TfidfVectorizer!
features_test = feature_extractor.transform(sotu_test['Nouns'])

Then we can use these features to predict the party affiliation for each speech:

In [ ]:
party_prediction = classifier.predict(features_test)
party_prediction

Since we know the true party affiliation of each speech's president, we can easily evaluate the accuracy of our predictions:

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_true=sotu_test['Party'], y_pred=party_prediction))

This little report tells us that the rules derived from the presidents in the training set held up in about 67 % of the presidents' addresses in the test set. We can also see that the tree is a little better at correctly identifying Democratic presidents than Republican ones. 

<div class="alert alert-block alert-info"> 

As we saw, using these very basic features and our very simple model, we cannot perfectly tell the two classes apart. We could of course consider to add other features or try out different models to improve the accuracy. But this analysis may also give us some ideas to help form hypotheses into what to look into next! For example, it may be worthwhile taking just some of the most salient terms at the top of tree (like *America*, *protection*, and *community*) and analyze their usage in the speeches over time and across the party affiliation.
</div>

## 4. Next steps

I hope this got you curious about how machine learning can add value to your work. 

If you want to continue with this dataset with a more advanced exercise, you could try to find out which speeches are the most similar using the [`KNearestNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier).

If you want to learn more about machine learning and how to identify the best parameters for a model, check out our [Intro to Machine Learning with scikit-learn](https://git.dartmouth.edu/lib-digital-strategies/RDS/workshops/machine-learning/intro-to-machine-learning-with-scikit-learn), if you haven't already. 

Finally, if you want to include machine learning in your work but don't really know how to get started, [reach out to us](mailto:researchdatahelp@groups.dartmouth.edu) and we will figure it out together!

## 
<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
